In [ ]:
using SemidefiniteOptInterface
const SDOI = SemidefiniteOptInterface

using Base.Test

using MathOptInterface
const MOI = MathOptInterface
const MOIT = MOI.Test
const MOIB = MOI.Bridges
const MOIU = MOI.Utilities

We hack the `optimize!` call of CSDP to print the `mock_optimize!` function to be used in the tests for the mock_optimizer.
The output still needs to be edited and replaced by the exact solution but this is a good start

In [ ]:
const MOIU = MOI.Utilities
MOIU.@model SDModelData () (EqualTo, GreaterThan, LessThan) (Zeros, Nonnegatives, Nonpositives, PositiveSemidefiniteConeTriangle) () (SingleVariable,) (ScalarAffineFunction,) (VectorOfVariables,) (VectorAffineFunction,)

In [ ]:
import CSDP
optimizer = CSDP.CSDPOptimizer(printlevel=0)

cached_optimizer = MOIU.CachingOptimizer(SDModelData{Float64}(), optimizer)
bridged_optimizer = MOIB.SplitInterval{Float64}(MOIB.RootDet{Float64}(MOIB.GeoMean{Float64}(MOIB.RSOCtoPSD{Float64}(MOIB.SOCtoPSD{Float64}(cached_optimizer)))))

config = MOIT.TestConfig(atol=1e-4, rtol=1e-4)

In [ ]:
function MOI.optimize!(m::SDOI.SOItoMOIBridge)
    MOI.optimize!(m.sdoptimizer)
    X = SDOI.getX(m.sdoptimizer)
    println("MOIU.set_mock_optimize!(mock, (mock) -> MOIU.mock_optimize!(mock,")
    print(  "                                                            ")
    if MOI.get(m, MOI.DualStatus()) == MOI.InfeasibilityCertificate
        print("tuple()")
    else
        if MOI.get(m, MOI.PrimalStatus()) == MOI.InfeasibilityCertificate
            print("(MOI.InfeasibilityCertificate, ")
        end
        print(  "[")
        for (blk, dim) in enumerate(m.blockdims)
            if blk > 1
                print(", ")
            end
            #print(round.(SDOI.block(X, blk), 4))
            print(SDOI.block(X, blk))
        end
        print("]")
        if MOI.get(m, MOI.PrimalStatus()) == MOI.InfeasibilityCertificate
            print(")")
        end
    end
    if MOI.get(m, MOI.PrimalStatus()) != MOI.InfeasibilityCertificate
        println(",")
        print(  "                                                            ")
        #print(round.(SDOI.gety(m.sdoptimizer), 4))
        print(SDOI.gety(m.sdoptimizer))
    end
    println("))")
end

In [ ]:
mock = SDOI.MockSDOptimizer{Float64}()
mock_optimizer = SDOI.SDOIOptimizer(mock, Float64)
cached_mock_optimizer = MOIU.CachingOptimizer(SDModelData{Float64}(), mock_optimizer)
config = MOIT.TestConfig(atol=1e-4, rtol=1e-4)

In [ ]:
function generate_mock_test(testfun)
    testfun(bridged_optimizer, config)
    testcall = replace(string(testfun), "MathOptInterface.Test", "MOIT")
    println("$testcall(cached_mock_optimizer, config)")
end

In [ ]:
generate_mock_test(MOIT.solve_singlevariable_obj)

In [ ]:
for testfun in map(p -> p.second, sort(collect(MOIT.psdttests), by=p->p.first))
    generate_mock_test(testfun)
end